In [16]:
import matplotlib.pyplot as plt
import numpy as np
import csv
import math
import random
import pandas as pd

from BSE2 import market_session

%reload_ext autoreload
%autoreload 2

## Helper functions:

In [2]:
#helper functions

# plot trades of a single experiment
def plot_trades(trial_id):
    prices_fname = trial_id + '_tape.csv'
    x = np.empty(0)
    y = np.empty(0)
    with open(prices_fname, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            time = float(row[1])
            price = float(row[2])
            x = np.append(x,time)
            y = np.append(y,price)

    plt.plot(x, y, 'x', color='black') 
    
# run an experiment n times and plot all trades
def n_runs_plot_trades(n, trial_id, start_time, end_time, traders_spec, order_sched):
    x = np.empty(0)
    y = np.empty(0)

    for i in range(n):
        trialId = trial_id + '_' + str(i)
        tdump = open(trialId + '_avg_balance.csv','w')

        market_session(trialId, start_time, end_time, traders_spec, order_sched, tdump, True, False)
        
        tdump.close()

        with open(trialId + '_tape.csv', newline='') as csvfile:
            reader = csv.reader(csvfile)
            for row in reader:
                time = float(row[1])
                price = float(row[2])
                x = np.append(x,time)
                y = np.append(y,price)

    plt.plot(x, y, 'x', color='black');

# !!! Don't use on it's own   
def getorderprice(i, sched, n, mode):
    pmin = min(sched[0][0], sched[0][1])
    pmax = max(sched[0][0], sched[0][1])
    prange = pmax - pmin
    stepsize = prange / (n - 1)
    halfstep = round(stepsize / 2.0)

    if mode == 'fixed':
        orderprice = pmin + int(i * stepsize)
    elif mode == 'jittered':
        orderprice = pmin + int(i * stepsize) + random.randint(-halfstep, halfstep)
    elif mode == 'random':
        if len(sched) > 1:
            # more than one schedule: choose one equiprobably
            s = random.randint(0, len(sched) - 1)
            pmin = min(sched[s][0], sched[s][1])
            pmax = max(sched[s][0], sched[s][1])
        orderprice = random.randint(pmin, pmax)
    return orderprice    

# !!! Don't use on it's own
def make_supply_demand_plot(bids, asks):
    # total volume up to current order
    volS = 0
    volB = 0

    fig, ax = plt.subplots()
    plt.ylabel('Price')
    plt.xlabel('Quantity')
    
    pr = 0
    for b in bids:
        if pr != 0:
            # vertical line
            ax.plot([volB,volB], [pr,b], 'r-')
        # horizontal lines
        line, = ax.plot([volB,volB+1], [b,b], 'r-')
        volB += 1
        pr = b
    if bids:
        line.set_label('Demand')
        
    pr = 0
    for s in asks:
        if pr != 0:
            # vertical line
            ax.plot([volS,volS], [pr,s], 'b-')
        # horizontal lines
        line, = ax.plot([volS,volS+1], [s,s], 'b-')
        volS += 1
        pr = s
    if asks:
        line.set_label('Supply')
        
    if bids or asks:
        plt.legend()
    plt.show()

# plot supply and demand curves from supply and demand ranges and stepmode
def plot_sup_dem(seller_num, sup_ranges, buyer_num, dem_ranges, stepmode):
    asks = []
    for s in range(seller_num):
        asks.append(getorderprice(s, sup_ranges, seller_num, stepmode))
    asks.sort()
    bids = []
    for b in range(buyer_num):
        bids.append(getorderprice(b, dem_ranges, buyer_num, stepmode))
    bids.sort()
    bids.reverse()
    
    make_supply_demand_plot(bids, asks) 

# plot sorted trades, useful is some situations - won't be used in this worksheet
def in_order_plot(trial_id):
    prices_fname = trial_id + '_tape.csv'
    y = np.empty(0)
    with open(prices_fname, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            price = float(row[2])
            y = np.append(y,price)
    y = np.sort(y)
    x = list(range(len(y)))

    plt.plot(x, y, 'x', color='black')   

# plot offset function
def plot_offset_fn(offset_fn, total_time_seconds):   
    x = list(range(total_time_seconds))
    offsets = []
    for i in range(total_time_seconds):
        offsets.append(offset_fn(i))
    plt.plot(x, offsets, 'x', color='black')  

## Homogenous Static markets


In [ ]:
sup_range = (60, 60)
dem_range = (140,140)

n_days = 3
start_time = 0
end_time = 60 * 60 * 24 * n_days

supply_schedule = [{'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}]
demand_schedule = [{'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}]

order_interval = 5
order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
               'interval': order_interval, 'timemode': 'drip-poisson'}


dump_all = True
verbose = True


for k in range(4,11):
    for f in np.arange(0.0,2.1, 0.1):
        buyers_spec = [('PRDE', 30, {'f_value': f, 'k': k, 's_min': -1.0, 's_max': +1.0})]
        sellers_spec = buyers_spec
        traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec}
        trial = 1
        while trial <= 5:
            trial_id = 'trial%02d_%.02f' % (k, f)
            balances_file = open(trial_id + '_avg_balance.csv', 'w')
            market_session(trial_id, start_time, end_time, traders_spec,
                           order_sched, balances_file, True, True)
            balances_file.close()
            trial = trial + 1




## Homogenous Dynamic

In [ ]:
sup_range1 = (60, 60)
sup_range2 = (100,100)
dem_range1 = (140,140)
dem_range2 = (180,180)


n_days = 3
start_time = 0
end_time = 60 * 60 * 24 * n_days


supply_schedule = [{'from':start_time, 'to':end_time//3, 'ranges':[sup_range1], 'stepmode':'fixed'},
                        {'from':end_time//3, 'to':2*end_time//3, 'ranges':[sup_range2], 'stepmode':'fixed'},
                        {'from':2*end_time//3, 'to':end_time, 'ranges':[sup_range1], 'stepmode':'fixed'}
                      ]
demand_schedule = [{'from':start_time, 'to':end_time//3, 'ranges':[dem_range1], 'stepmode':'fixed'},
                        {'from':end_time//3, 'to':2*end_time//3, 'ranges':[dem_range2], 'stepmode':'fixed'},
                        {'from':2*end_time//3, 'to':end_time, 'ranges':[dem_range1], 'stepmode':'fixed'}
                      ]


order_interval = 5
order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
               'interval': order_interval, 'timemode': 'drip-poisson'}


dump_all = False
verbose = False

combinations = [(4,2),(5,2),(4,1.6), (7,2), (5,1.8), (4,0.2), (4,0.1), (4,0.7), (6,0.4), (4,0.5), (5,1.9), (6,2), (4,1.7)]

for (k,f) in combinations:
    buyers_spec = [('PRDE', 30, {'f_value': f, 'k': k, 's_min': -1.0, 's_max': +1.0})]
    sellers_spec = buyers_spec
    traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec}
    trial = 1
    while trial <= 5:
        trial_id = 'trial%02d_%.02f_%02d' % (k, f, trial)
        balances_file = open(trial_id + '_avg_balance.csv', 'w')
        market_session(trial_id, start_time, end_time, traders_spec,
                       order_sched, balances_file, True, True)
        balances_file.close()
        trial = trial + 1

## Homogenous Static Retry

In [ ]:
sup_range = (60, 60)
dem_range = (140,140)


n_days = 3
start_time = 0
end_time = 60 * 60 * 24 * n_days


supply_schedule = [{'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}]
demand_schedule = [{'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}]



order_interval = 5
order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
               'interval': order_interval, 'timemode': 'drip-poisson'}


dump_all = False
verbose = False

combinations = [(4,2),(5,2),(4,1.6), (7,2), (5,1.8), (4,0.2), (4,0.1), (4,0.7), (6,0.4), (4,0.5), (5,1.9), (6,2), (4,1.7)]

for (k,f) in combinations:
    buyers_spec = [('PRDE', 30, {'f_value': f, 'k': k, 's_min': -1.0, 's_max': +1.0})]
    sellers_spec = buyers_spec
    traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec}
    trial = 1
    while trial <= 4:
        trial_id = 'static_homog_retry/trial%02d_%.02f_%02d' % (k, f, trial)
        balances_file = open(trial_id + '_avg_balance.csv', 'w')
        market_session(trial_id, start_time, end_time, traders_spec,
                       order_sched, balances_file, True, True)
        balances_file.close()
        trial = trial + 1

## Static balanced


In [ ]:
sup_range = (60, 60)
dem_range = (140,140)


n_days = 3
start_time = 0
end_time = 60 * 60 * 24 * n_days


supply_schedule = [{'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}]
demand_schedule = [{'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}]



order_interval = 5
order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
               'interval': order_interval, 'timemode': 'drip-poisson'}


dump_all = False
verbose = False

best_buyers = [(4,0.2), (4,0.7)]
best_sellers = [(4,1.7), (4,2)]

comb = [best_buyers, best_sellers]


k1,f1 = best_buyers[0]
k2,f2 = best_buyers[1]


buyers_spec = [('PRDE', 15, {'f_value': f1, 'k': k1, 's_min': -1.0, 's_max': +1.0}), ('PRDE', 15, {'f_value': f2, 'k': k2, 's_min': -1.0, 's_max': +1.0})]
sellers_spec = [('PRDE', 15, {'f_value': f1, 'k': k1, 's_min': -1.0, 's_max': +1.0}), ('PRDE', 15, {'f_value': f2, 'k': k2, 's_min': -1.0, 's_max': +1.0})]
traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec}
trial = 1
while trial <= 5:
    trial_id = 'best2_static/trialbuyers_%02d' % (trial)
    balances_file = open(trial_id + '_avg_balance.csv', 'w')
    market_session(trial_id, start_time, end_time, traders_spec,
                   order_sched, balances_file, True, True)
    balances_file.close()
        trial = trial + 1

## static one-in-many

In [ ]:
sup_range = (60, 60)
dem_range = (140,140)


n_days = 3
start_time = 0
end_time = 60 * 60 * 24 * n_days


supply_schedule = [{'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}]
demand_schedule = [{'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}]



order_interval = 5
order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
               'interval': order_interval, 'timemode': 'drip-poisson'}


dump_all = False
verbose = False


combinations = [(4,2),(5,2),(4,1.6), (7,2), (4,0.2), (4,0.1), (4,0.7), (6,0.4), (4,0.5), (5,1.9), (6,2), (4,1.7), (5,1.8)]


for (k,f) in combinations:
    sellers_spec = [('PRDE', 29, {'f_value': 0.8, 'k': 4, 's_min': -1.0, 's_max': +1.0}), ('PRDE', 1, {'f_value': f, 'k': k, 's_min': -1.0, 's_max': +1.0})]
    buyers_spec = [('PRDE', 30, {'f_value': 0.8, 'k': 4, 's_min': -1.0, 's_max': +1.0})]
    traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec}
    trial = 1
    while trial <= 5:
        trial_id = 'seller_one_in_many_static/trial%02d_%.02f_%02d' % (k, f, trial)
        balances_file = open(trial_id + '_avg_balance.csv', 'w')
        market_session(trial_id, start_time, end_time, traders_spec,
                       order_sched, balances_file, True, True)
        balances_file.close()
        trial = trial + 1

In [ ]:
#static one in many buyer
sup_range = (60, 60)
dem_range = (140,140)


n_days = 3
start_time = 0
end_time = 60 * 60 * 24 * n_days


supply_schedule = [{'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}]
demand_schedule = [{'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}]



order_interval = 5
order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
               'interval': order_interval, 'timemode': 'drip-poisson'}


dump_all = False
verbose = False


combinations = [(4,2),(5,2),(4,1.6), (7,2),(5,1.8), (4,0.2), (4,0.1), (4,0.7), (6,0.4), (4,0.5), (5,1.9), (6,2), (4,1.7)]


for (k,f) in combinations:
    buyers_spec = [('PRDE', 29, {'f_value': 0.8, 'k': 4, 's_min': -1.0, 's_max': +1.0}), ('PRDE', 1, {'f_value': f, 'k': k, 's_min': -1.0, 's_max': +1.0})]
    sellers_spec = [('PRDE', 30, {'f_value': 0.8, 'k': 4, 's_min': -1.0, 's_max': +1.0})]
    traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec}
    trial = 1
    while trial <= 5:
        trial_id = 'one_in_many_static/trial%02d_%.02f_%02d' % (k, f, trial)
        balances_file = open(trial_id + '_avg_balance.csv', 'w')
        market_session(trial_id, start_time, end_time, traders_spec,
                       order_sched, balances_file, True, True)
        balances_file.close()
        trial = trial + 1

In [ ]:
#dynamic one in many buyer
sup_range1 = (60, 60)
sup_range2 = (100,100)
dem_range1 = (140,140)
dem_range2 = (180,180)


n_days = 3
start_time = 0
end_time = 60 * 60 * 24 * n_days


supply_schedule = [{'from':start_time, 'to':end_time//3, 'ranges':[sup_range1], 'stepmode':'fixed'},
                        {'from':end_time//3, 'to':2*end_time//3, 'ranges':[sup_range2], 'stepmode':'fixed'},
                        {'from':2*end_time//3, 'to':end_time, 'ranges':[sup_range1], 'stepmode':'fixed'}
                      ]
demand_schedule = [{'from':start_time, 'to':end_time//3, 'ranges':[dem_range1], 'stepmode':'fixed'},
                        {'from':end_time//3, 'to':2*end_time//3, 'ranges':[dem_range2], 'stepmode':'fixed'},
                        {'from':2*end_time//3, 'to':end_time, 'ranges':[dem_range1], 'stepmode':'fixed'}
                      ]



order_interval = 5
order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
               'interval': order_interval, 'timemode': 'drip-poisson'}


dump_all = False
verbose = False


combinations = [(4,2),(5,2),(4,1.6), (7,2),(5,1.8), (4,0.2), (4,0.1), (4,0.7), (6,0.4), (4,0.5), (5,1.9), (6,2), (4,1.7)]


for (k,f) in combinations:
    buyers_spec = [('PRDE', 29, {'f_value': 0.8, 'k': 4, 's_min': -1.0, 's_max': +1.0}), ('PRDE', 1, {'f_value': f, 'k': k, 's_min': -1.0, 's_max': +1.0})]
    sellers_spec = [('PRDE', 30, {'f_value': 0.8, 'k': 4, 's_min': -1.0, 's_max': +1.0})]
    traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec}
    trial = 1
    while trial <= 5:
        trial_id = 'one_in_many_dynamic_buyer/trial%02d_%.02f_%02d' % (k, f, trial)
        balances_file = open(trial_id + '_avg_balance.csv', 'w')
        market_session(trial_id, start_time, end_time, traders_spec,
                       order_sched, balances_file, True, True)
        balances_file.close()
        trial = trial + 1

In [ ]:
#dynamic one in many seller
sup_range = (60, 60)
dem_range = (140,140)


n_days = 3
start_time = 0
end_time = 60 * 60 * 24 * n_days


supply_schedule = [{'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}]
demand_schedule = [{'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}]



order_interval = 5
order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
               'interval': order_interval, 'timemode': 'drip-poisson'}


dump_all = False
verbose = False


combinations = [(4,2),(5,2),(4,1.6), (7,2),(5,1.8), (4,0.2), (4,0.1), (4,0.7), (6,0.4), (4,0.5), (5,1.9), (6,2), (4,1.7)]


for (k,f) in combinations:
    sellers_spec = [('PRDE', 29, {'f_value': 0.8, 'k': 4, 's_min': -1.0, 's_max': +1.0}), ('PRDE', 1, {'f_value': f, 'k': k, 's_min': -1.0, 's_max': +1.0})]
    buyers_spec = [('PRDE', 30, {'f_value': 0.8, 'k': 4, 's_min': -1.0, 's_max': +1.0})]
    traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec}
    trial = 1
    while trial <= 1:
        trial_id = 'one_in_many_dynamic_seller/trial%02d_%.02f_%02d' % (k, f, trial)
        balances_file = open(trial_id + '_avg_balance.csv', 'w')
        market_session(trial_id, start_time, end_time, traders_spec,
                       order_sched, balances_file, True, True)
        balances_file.close()
        trial = trial + 1

In [ ]:
# dynamic balanced hetero
sup_range1 = (60, 60)
sup_range2 = (100,100)
dem_range1 = (140,140)
dem_range2 = (180,180)


n_days = 3
start_time = 0
end_time = 60 * 60 * 24 * n_days


supply_schedule = [{'from':start_time, 'to':end_time//3, 'ranges':[sup_range1], 'stepmode':'fixed'},
                        {'from':end_time//3, 'to':2*end_time//3, 'ranges':[sup_range2], 'stepmode':'fixed'},
                        {'from':2*end_time//3, 'to':end_time, 'ranges':[sup_range1], 'stepmode':'fixed'}
                      ]
demand_schedule = [{'from':start_time, 'to':end_time//3, 'ranges':[dem_range1], 'stepmode':'fixed'},
                        {'from':end_time//3, 'to':2*end_time//3, 'ranges':[dem_range2], 'stepmode':'fixed'},
                        {'from':2*end_time//3, 'to':end_time, 'ranges':[dem_range1], 'stepmode':'fixed'}
                      ]




order_interval = 5
order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
               'interval': order_interval, 'timemode': 'drip-poisson'}


dump_all = False
verbose = False

# balanced


# combinations = [(4,0.2),(4,0.1),(4,0.7), (6,0.4), (4,0.5)]
combinations = [(4,2),(5,2),(4,1.6), (7,2), (5,1.8), (5,1.9), (6,2), (4,1.7)]

for i in range(0,len(combinations),2):
    k1, f1 = combinations[i]
    k2, f2 = combinations[i+1]
    
    
   
    buyers_spec = [('PRDE', 15, {'f_value': 4, 'k': k1, 's_min': -1.0, 's_max': +1.0}), ('PRDE', 15, {'f_value': f2, 'k': k2, 's_min': -1.0, 's_max': +1.0})]
    sellers_spec = buyers_spec
    traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec}
    trial = 1
    while trial <= 5:
        trial_id = 'dyn_balanced/trial%02d_%.02f_%02d' % (k, f, trial)
        balances_file = open(trial_id + '_avg_balance.csv', 'w')
        market_session(trial_id, start_time, end_time, traders_spec,
                       order_sched, balances_file, True, True)
        balances_file.close()
        trial = trial + 1

In [ ]:
# static hetero
sup_range = (60, 60)
dem_range = (140,140)


n_days = 3
start_time = 0
end_time = 60 * 60 * 24 * n_days


supply_schedule = [{'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}]
demand_schedule = [{'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}]



order_interval = 5
order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
               'interval': order_interval, 'timemode': 'drip-poisson'}


dump_all = False
verbose = False

# balanced static hetero


# combinations = [(4,0.2),(4,0.1),(4,0.7), (6,0.4), (4,0.5)]
combinations = [(4,2),(5,2),(4,1.6), (7,2), (5,1.8), (5,1.9), (6,2), (4,1.7)]

for i in range(0,len(combinations),2):
    k1, f1 = combinations[i]
    k2, f2 = combinations[i+1]
    
    
   
    buyers_spec = [('PRDE', 15, {'f_value': 4, 'k': k1, 's_min': -1.0, 's_max': +1.0}), ('PRDE', 15, {'f_value': f2, 'k': k2, 's_min': -1.0, 's_max': +1.0})]
    sellers_spec = buyers_spec
    traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec}
    trial = 1
    while trial <= 2:
        trial_id = 'static_hetero/trial%02d_%.02f_%02d' % (k, f, trial)
        balances_file = open(trial_id + '_avg_balance.csv', 'w')
        market_session(trial_id, start_time, end_time, traders_spec,
                       order_sched, balances_file, True, True)
        balances_file.close()
        trial = trial + 1

## PRDE-best trials


In [ ]:
sup_range = (60, 60)
dem_range = (140,140)


n_days = 3
start_time = 0
end_time = 60 * 60 * 24 * n_days


supply_schedule = [{'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}]
demand_schedule = [{'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}]



order_interval = 5
order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
               'interval': order_interval, 'timemode': 'drip-poisson'}


dump_all = False
verbose = False

best_buyers = [(5,2), (4,0.1)]
best_sellers = [(4,2), (6,2)]

comb = [best_buyers, best_sellers]


for pair in comb:
    k1,f1 = pair[0]
    k2,f2 = pair[1]
    

    buyers_spec = [('PRDE', 15, {'f_value': f1, 'k': k1, 's_min': -1.0, 's_max': +1.0}), ('PRDE', 15, {'f_value': f2, 'k': k2, 's_min': -1.0, 's_max': +1.0})]
    sellers_spec = [('PRDE', 15, {'f_value': f1, 'k': k1, 's_min': -1.0, 's_max': +1.0}), ('PRDE', 15, {'f_value': f2, 'k': k2, 's_min': -1.0, 's_max': +1.0})]
    traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec}
    trial = 1
    while trial <= 5:
        trial_id = 'best2_dynamic/trialbuyers_%02d' % (trial)
        balances_file = open(trial_id + '_avg_balance.csv', 'w')
        market_session(trial_id, start_time, end_time, traders_spec,
                       order_sched, balances_file, True, True)
        balances_file.close()
        trial = trial + 1

In [ ]:
# static balanced
sup_range = (60, 60)
dem_range = (140,140)


n_days = 3
start_time = 0
end_time = 60 * 60 * 24 * n_days


supply_schedule = [{'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}]
demand_schedule = [{'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}]



order_interval = 5
order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
               'interval': order_interval, 'timemode': 'drip-poisson'}


dump_all = False
verbose = False

best_buyers = [(4,0.2), (4,0.7)]
best_sellers = [(4,1.7), (4,2)]

comb = [best_buyers, best_sellers]


k1,f1 = best_buyers[0]
k2,f2 = best_buyers[1]


buyers_spec = [('PRDE', 15, {'f_value': f1, 'k': k1, 's_min': -1.0, 's_max': +1.0}), ('PRDE', 15, {'f_value': f2, 'k': k2, 's_min': -1.0, 's_max': +1.0})]
sellers_spec = [('PRDE', 15, {'f_value': f1, 'k': k1, 's_min': -1.0, 's_max': +1.0}), ('PRDE', 15, {'f_value': f2, 'k': k2, 's_min': -1.0, 's_max': +1.0})]
traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec}
trial = 1
while trial <= 5:
    trial_id = 'best2_static/trialbuyers_%02d' % (trial)
    balances_file = open(trial_id + '_avg_balance.csv', 'w')
    market_session(trial_id, start_time, end_time, traders_spec,
                   order_sched, balances_file, True, True)
    balances_file.close()
    trial = trial + 1

In [ ]:
# prde-best homog static
sup_range = (60, 60)
dem_range = (140,140)


n_days = 3
start_time = 0
end_time = 60 * 60 * 24 * n_days


supply_schedule = [{'from': start_time, 'to': end_time, 'ranges': [sup_range], 'stepmode': 'fixed'}]
demand_schedule = [{'from': start_time, 'to': end_time, 'ranges': [dem_range], 'stepmode': 'fixed'}]



order_interval = 5
order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
               'interval': order_interval, 'timemode': 'drip-poisson'}


dump_all = False
verbose = False

combinations = [(4,2),(5,2),(4,1.6), (7,2), (5,1.8), (4,0.2), (4,0.1), (4,0.7), (6,0.4), (4,0.5), (5,1.9), (6,2), (4,1.7)]
combinations = [(4,0.2), (4,0.1), (4,2.0),(5,1.9) ]

for (k,f) in combinations:
    buyers_spec = [('PRDE-best', 30, {'f_value': f, 'k': k, 's_min': -1.0, 's_max': +1.0})]
    sellers_spec = buyers_spec
    traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec}
    trial = 1
    while trial <= 5:
        trial_id = 'prde-best_homog_stat/trial%02d_%.02f_%02d' % (k, f, trial)
        balances_file = open(trial_id + '_avg_balance.csv', 'w')
        market_session(trial_id, start_time, end_time, traders_spec,
                       order_sched, balances_file, True, True)
        balances_file.close()
        trial = trial + 1

In [ ]:
# prde-best homog dynamic
sup_range1 = (60, 60)
sup_range2 = (100,100)
dem_range1 = (140,140)
dem_range2 = (180,180)


n_days = 3
start_time = 0
end_time = 60 * 60 * 24 * n_days


supply_schedule = [{'from':start_time, 'to':end_time//3, 'ranges':[sup_range1], 'stepmode':'fixed'},
                        {'from':end_time//3, 'to':2*end_time//3, 'ranges':[sup_range2], 'stepmode':'fixed'},
                        {'from':2*end_time//3, 'to':end_time, 'ranges':[sup_range1], 'stepmode':'fixed'}
                      ]
demand_schedule = [{'from':start_time, 'to':end_time//3, 'ranges':[dem_range1], 'stepmode':'fixed'},
                        {'from':end_time//3, 'to':2*end_time//3, 'ranges':[dem_range2], 'stepmode':'fixed'},
                        {'from':2*end_time//3, 'to':end_time, 'ranges':[dem_range1], 'stepmode':'fixed'}
                      ]


order_interval = 5
order_sched = {'sup': supply_schedule, 'dem': demand_schedule,
               'interval': order_interval, 'timemode': 'drip-poisson'}


dump_all = False
verbose = False

# combinations = [(4,2),(5,2),(4,1.6), (7,2), (5,1.8), (4,0.2), (4,0.1), (4,0.7), (6,0.4), (4,0.5), (5,1.9), (6,2), (4,1.7)]
combinations = [(5,2), (4,0.1),(4,2), (6,2)]


for (k,f) in combinations:
    buyers_spec = [('PRDE-best', 30, {'f_value': f, 'k': k, 's_min': -1.0, 's_max': +1.0})]
    sellers_spec = buyers_spec
    traders_spec = {'sellers': sellers_spec, 'buyers': buyers_spec}
    trial = 1
    while trial <= 5:
        trial_id = 'prde-best_homog_dy/trial%02d_%.02f_%02d' % (k, f, trial)
        balances_file = open(trial_id + '_avg_balance.csv', 'w')
        market_session(trial_id, start_time, end_time, traders_spec,
                       order_sched, balances_file, True, True)
        balances_file.close()
        trial = trial + 1